Jupyter Notebook 執行版：https://mybinder.org/v2/gh/nandemoi/test/HEAD

先定義一個 dictionary

In [1]:
d = { "a" : 1, "b" : 2, "c" : 3}

一一檢視 dictionary 內容的方法方法 1:

In [2]:
for key in d:
    print (key, d [key])

a 1
b 2
c 3


一一檢視 dictionary 內容的方法方法 2:

In [3]:
for key, value in d.items():
    print (key, value)

a 1
b 2
c 3


如何在 dictionary 中加入新的 key/value

In [4]:
d [ "e" ] = 5
print (d)

{'a': 1, 'b': 2, 'c': 3, 'e': 5}


要找 Youbike「植物園站」有幾輛車

In [5]:
# 先匯入要用到的模組
import requests
import time

循序暴力窮舉搜尋

In [6]:
def prBikeNum (stName): # 回傳搜尋部分花多少微秒
    # 抓最新站況
    ubikes = requests.get ("https://tcgbusfs.blob.core.windows.net/blobyoubike/YouBikeTP.json").json()["retVal"]
    start_time = time.time() # 開始搜尋的時間點
    for stNo, stats in ubikes.items ():
        if stats ['sna'] == stName:
            print (stats ['bemp'])
            return int ((time.time() - start_time)*1e6)
    print ("站名錯誤")
    return int ((time.time() - start_time)*1e6)

In [7]:
t = prBikeNum ("植物園")
print("--- 搜尋使用 %s 微秒 ---" % t)
t = prBikeNum ("重慶南海路口")
print("--- 搜尋使用 %s 微秒 ---" % t)

35
--- 搜尋使用 87 微秒 ---
17
--- 搜尋使用 439 微秒 ---


如果站名不存在，要花多少時間？ O (n)

In [8]:
t = prBikeNum ('南海重慶路口')
print("--- 搜尋使用 %s 微秒 ---" % t)

站名錯誤
--- 搜尋使用 170 微秒 ---


更有效率的演算法：<br>
先建立站名到站號的對應關係 (用 dictionary; 因為這個關係不會變，所以做一次就好)

In [9]:
ubikes = requests.get ("https://tcgbusfs.blob.core.windows.net/blobyoubike/YouBikeTP.json").json()["retVal"]
name2no = { }
for stNo, stats in ubikes.items (): # no 是站號, stat 是站況
    stName = stats [ 'sna' ] # 每一站的站名
    name2no [ stName ] = stNo # 編一本從站名查站號的 dictionary name2no 
    # 比較厲害的寫法，上面 2 行寫成 1 行：
    # name2no [ stat [ 'sna' ] ] = key

連同原來的 ubikes，現在有 2 個表<br>
<img src="http://nandemoi.github.io/tables.png" height="600px"><br>
之後查車數查表 2 次就有答案了。 O (1) + O (1) = O (1)

In [10]:
def prBikeNum1 (stName): # 回傳搜尋部分花多少微秒
    # 抓最新站況
    ubikes = requests.get ("https://tcgbusfs.blob.core.windows.net/blobyoubike/YouBikeTP.json").json()["retVal"]
    start_time = time.time() # 開始搜尋的時間點
    try:
        stNo = name2no [ stName ] # 查站名 stName 所對應的站號
        stats = ubikes [ stNo ]    # 查站號 no 的站況
        print (stats ['bemp'])
        # 比較厲害的寫法，上面 3 行寫成 1 行
        # print (ubikes [ name2no [ stName ] ]['bemp']) 
    except: # key error
        print ("站名錯誤")
    finally:
        return int ((time.time() - start_time)*1e6)

In [11]:
t = prBikeNum1 ("植物園")
print("--- 搜尋使用 %s 微秒 ---" % t)
t = prBikeNum1 ("重慶南海路口")
print("--- 搜尋使用 %s 微秒 ---" % t)

35
--- 搜尋使用 78 微秒 ---
17
--- 搜尋使用 137 微秒 ---


如果站名不存在，要花多少時間？ O (1)

In [12]:
t = prBikeNum1 ('南海重慶路口')
print("--- 搜尋使用 %s 微秒 ---" % t)

站名錯誤
--- 搜尋使用 82 微秒 ---


捲回去比較一下 !!!

可能有同學迫不及待想嗆老師：整體又差不了多少！是沒有錯，因為現在 CPU 速度都很快，整個操作的瓶頸在網路資料的接收，後面的搜尋相較起來不成比例。<br>
所以請思考 2 個問題：<br>
(1) 會不會計算硬體效能越來越好，研究有效率的演算法越來越不重要？新的計算硬體科技興起會不會淘汰舊的運算思維模式？<br>
(2) 通訊是操作的瓶頸，如果通訊科技變快了 (例如 5G)，那瓶頸是不是還在網路？<br>
答案就留給同學自己在歷程發揮，我就不明講。台北市 Youbikes 只有幾百個站，如果你的資料量大到呈指數倍成長時，問題還是一樣嗎？<br>
<br>
半個學期的專題能夠完成的有限，這個價值是我認為同學可以在歷程中呈現的。本來想讓同學自己想出來，不過好像不少同學可以從這樣的帶領獲得一點認識。

同場加映：<br>
下拉式選單 https://stackoverflow.com/questions/34020789/ipywidgets-dropdown-widgets-what-is-the-onchange-event

In [13]:
import ipywidgets as wgts

stName = wgts.Dropdown(
    options=['植物園', '重慶南海路口', '和平重慶路口', '臺北市立大學'],
    value='重慶南海路口',
    description='請選擇:',
)
display (stName)

Dropdown(description='請選擇:', index=1, options=('植物園', '重慶南海路口', '和平重慶路口', '臺北市立大學'), value='重慶南海路口')

選定上面的選單後，再執行下面的程式塊

In [14]:
t = prBikeNum1 (stName.value)
print("--- 搜尋使用 %s 微秒 ---" % t)

17
--- 搜尋使用 149 微秒 ---
